In [1]:
# General imports
import itertools
import numpy as np
import pandas as pd
import cudf
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

## Function

In [2]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [3]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [ ]:
def create_level(data, by, level_num):
    
    index_col = ['id','item_id','dept_id','cat_id','state_id','store_id']
    
    for col in by:
        index_col.remove(col)
        
    level = data.groupby(by).sum().reset_index()
    data_len = np.prod([data[col].unique().shape[0] for col in by])
    
    ids = [data[col].unique() for col in by]
    ids = list(itertools.product(*ids))
    ids = sorted(['_'.join(i) for i in ids])
    
    index_data = []
    for j, val_j in enumerate(index_col):
        col = []
        for i, val_i in enumerate(ids):
            unique_values = data[val_j].unique()
            for val_i_modified in val_i.split('_'):
                if val_i_modified not in ['1','2','3','4']:
                    contains = unique_values[unique_values.str.contains(val_i_modified)].tolist()
                    if contains:
                        break
            
            if contains and val_j != 'id' and val_j != 'item_id':
                col.append('_'.join(contains))
            elif val_j == 'id':
                col.append('level' + level_num + '_' + val_i + '_' + 'validation')
            elif val_j == 'item_id':
                col.append('all' + '_' + val_i)
            else:
                col.append('_'.join(unique_values.tolist()))
        index_data.append(col)
    
    level[index_col] = pd.DataFrame(index_data).T
    return level

## Sales Data

### Loading sales df to create levels

In [7]:
train_df = cudf.read_csv('data/sales_train_validation.csv')

### Calculating Levels

In [8]:
train_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [9]:
train_df.id.value_counts()

FOODS_1_001_CA_1_validation        1
FOODS_1_001_CA_2_validation        1
FOODS_1_001_CA_3_validation        1
FOODS_1_001_CA_4_validation        1
FOODS_1_001_TX_1_validation        1
                                  ..
HOUSEHOLD_2_516_TX_2_validation    1
HOUSEHOLD_2_516_TX_3_validation    1
HOUSEHOLD_2_516_WI_1_validation    1
HOUSEHOLD_2_516_WI_2_validation    1
HOUSEHOLD_2_516_WI_3_validation    1
Name: id, Length: 30490, dtype: int32

#### Level 1

In [10]:
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']

In [11]:
level1 = train_df.sum().to_frame().T

ValueError: could not convert string to float: 'HOBBIES_1_001_CA_1_validationHOBBIES_1_002_CA_1_validationHOBBIES_1_003_CA_1_validationHOBBIES_1_004_CA_1_validationHOBBIES_1_005_CA_1_validationHOBBIES_1_006_CA_1_validationHOBBIES_1_007_CA_1_validationHOBBIES_1_008_CA_1_validationHOBBIES_1_009_CA_1_validationHOBBIES_1_010_CA_1_validationHOBBIES_1_011_CA_1_validationHOBBIES_1_012_CA_1_validationHOBBIES_1_013_CA_1_validationHOBBIES_1_014_CA_1_validationHOBBIES_1_015_CA_1_validationHOBBIES_1_016_CA_1_validationHOBBIES_1_017_CA_1_validationHOBBIES_1_018_CA_1_validationHOBBIES_1_019_CA_1_validationHOBBIES_1_020_CA_1_validationHOBBIES_1_021_CA_1_validationHOBBIES_1_022_CA_1_validationHOBBIES_1_023_CA_1_validationHOBBIES_1_024_CA_1_validationHOBBIES_1_025_CA_1_validationHOBBIES_1_026_CA_1_validationHOBBIES_1_027_CA_1_validationHOBBIES_1_028_CA_1_validationHOBBIES_1_029_CA_1_validationHOBBIES_1_030_CA_1_validationHOBBIES_1_031_CA_1_validationHOBBIES_1_032_CA_1_validationHOBBIES_1_033_CA_1_valid

In [ ]:
'_'.join(train_df.cat_id.unique())

In [12]:
level1[index_columns] = ['level_1_validation',
                         'all',
                         '_'.join(train_df.dept_id.unique()),
                         '_'.join(train_df.cat_id.unique()),
                         '_'.join(train_df.store_id.unique()),
                         '_'.join(train_df.state_id.unique())]

NameError: name 'level1' is not defined

In [14]:
level1

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,level_1_validation,all,HOBBIES_1_HOBBIES_2_HOUSEHOLD_1_HOUSEHOLD_2_FO...,HOBBIES_HOUSEHOLD_FOODS,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,CA_TX_WI,32631,31749,23783,25412,...,41789,48362,51640,38059,37570,35343,35033,40517,48962,49795


#### Level 2

In [ ]:
level2 = create_level(train_df, ['state_id'], '2')
level2

#### Level 3

In [116]:
level3 = create_level_all_products(train_df, ['store_id'], '3')
level3

,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1909,d_1910,d_1911,d_1912,d_1913,id,item_id,dept_id,cat_id,state_id
0,CA_1,4337,4155,2816,3051,2630,3276,3450,5437,4340,...,3722,3709,4387,5577,6113,level_3_CA_1_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
1,CA_2,3494,3046,2121,2324,1942,2288,2629,3729,2957,...,3691,3303,4457,5884,6082,level_3_CA_2_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
2,CA_3,4739,4827,3785,4232,3817,4369,4703,5456,5581,...,5235,5018,5623,7419,7721,level_3_CA_3_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
3,CA_4,1625,1777,1386,1440,1536,1389,1469,1988,1818,...,2500,2458,2628,2954,3271,level_3_CA_4_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
4,TX_1,2556,2687,1822,2258,1694,2734,1691,2820,2887,...,2901,2776,3022,3700,4033,level_3_TX_1_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
5,TX_2,3852,3937,2731,2954,2492,3439,2588,3772,3657,...,3384,3446,3902,4483,4292,level_3_TX_2_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
6,TX_3,3030,3006,2225,2169,1726,2833,1947,2848,2832,...,3315,3380,3691,4083,3957,level_3_TX_3_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
7,WI_1,2704,2194,1562,1251,2,2049,2815,3248,1674,...,3242,3324,3991,4772,4874,level_3_WI_1_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
8,WI_2,2256,1922,2018,2522,1175,2244,2232,2643,2140,...,4194,4393,4988,5404,5127,level_3_WI_2_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
9,WI_3,4038,4198,3317,3211,2132,4590,4486,5991,4850,...,3159,3226,3828,4686,4325,level_3_WI_3_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI


#### Level 4

In [117]:
level4 = create_level_all_products(train_df, ['cat_id'], '4')
level4

,cat_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1909,d_1910,d_1911,d_1912,d_1913,id,item_id,dept_id,state_id,store_id
0,FOODS,23178,22758,17174,18878,14603,22093,20490,27751,24862,...,23632,23317,26704,31927,32654,level_4_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
1,HOBBIES,3764,3357,2682,2669,1814,3220,2944,3986,2899,...,3472,3353,4085,4787,4683,level_4_HOBBIES_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
2,HOUSEHOLD,5689,5634,3927,3865,2729,3898,4576,6195,4975,...,8239,8363,9728,12248,12458,level_4_HOUSEHOLD_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3


#### Level 5

In [118]:
level5 = create_level_all_products(train_df, ['dept_id'], '5')
level5

,dept_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1909,d_1910,d_1911,d_1912,d_1913,id,item_id,cat_id,state_id,store_id
0,FOODS_1,2343,2216,1657,1508,1209,1897,1903,2235,1925,...,2657,2639,3169,3712,3185,level_5_FOODS_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
1,FOODS_2,4094,4209,3174,3606,2869,4375,3349,4384,4273,...,4734,4446,4974,5524,6473,level_5_FOODS_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
2,FOODS_3,16741,16333,12343,13764,10525,15821,15238,21132,18664,...,16241,16232,18561,22691,22996,level_5_FOODS_3_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3,HOBBIES_1,3610,3172,2497,2531,1714,3133,2855,3831,2741,...,3141,3014,3678,4397,4226,level_5_HOBBIES_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
4,HOBBIES_2,154,185,185,138,100,87,89,155,158,...,331,339,407,390,457,level_5_HOBBIES_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
5,HOUSEHOLD_1,4105,3858,2827,2732,1802,2664,3161,4334,3494,...,6393,6580,7762,9545,9795,level_5_HOUSEHOLD_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
6,HOUSEHOLD_2,1584,1776,1100,1133,927,1234,1415,1861,1481,...,1846,1783,1966,2703,2663,level_5_HOUSEHOLD_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3


#### Level 6

In [119]:
level6 = create_level_all_products(train_df, ['state_id','cat_id'], '6')
level6

,state_id,cat_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id,item_id,dept_id,store_id
0,CA,FOODS,10101,9862,6944,7864,7178,8256,9005,11870,...,10088,9863,9479,10991,13845,14806,level_6_CA_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
1,CA,HOBBIES,1802,1561,1472,1405,1181,1459,1314,1986,...,1849,1646,1590,2015,2367,2357,level_6_CA_HOBBIES_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
2,CA,HOUSEHOLD,2292,2382,1692,1778,1566,1607,1932,2754,...,3847,3639,3419,4089,5622,6024,level_6_CA_HOUSEHOLD_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3,TX,FOODS,6853,7030,5124,5470,4602,7067,4671,7055,...,7066,6242,6167,6839,7849,7765,level_6_TX_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
4,TX,HOBBIES,879,870,526,809,501,831,390,785,...,1025,980,882,1056,1222,1266,level_6_TX_HOBBIES_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
5,TX,HOUSEHOLD,1706,1730,1128,1102,809,1108,1165,1600,...,2831,2378,2553,2720,3195,3251,level_6_TX_HOUSEHOLD_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
6,WI,FOODS,6224,5866,5106,5544,2823,6770,6814,8826,...,7794,7527,7671,8874,10233,10083,level_6_WI_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
7,WI,HOBBIES,1083,926,684,455,132,930,1240,1215,...,913,846,881,1014,1198,1060,level_6_WI_HOBBIES_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
8,WI,HOUSEHOLD,1691,1522,1107,985,354,1183,1479,1841,...,2157,2222,2391,2919,3431,3183,level_6_WI_HOUSEHOLD_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3


#### Level 7

In [120]:
level7 = create_level_all_products(train_df, ['state_id','dept_id'], '7')
level7

,state_id,dept_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id,item_id,cat_id,store_id
0,CA,FOODS_1,1157,1142,787,743,729,796,913,1036,...,1119,1216,1097,1474,1802,1529,level_7_CA_FOODS_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
1,CA,FOODS_2,1864,1987,1315,1452,1239,1427,1554,1962,...,1753,1753,1554,1821,2284,2836,level_7_CA_FOODS_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
2,CA,FOODS_3,7080,6733,4842,5669,5210,6033,6538,8872,...,7216,6894,6828,7696,9759,10441,level_7_CA_FOODS_3_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3,CA,HOBBIES_1,1739,1502,1422,1352,1139,1427,1281,1920,...,1717,1508,1457,1838,2203,2135,level_7_CA_HOBBIES_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
4,CA,HOBBIES_2,63,59,50,53,42,32,33,66,...,132,138,133,177,164,222,level_7_CA_HOBBIES_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
5,CA,HOUSEHOLD_1,1527,1515,1116,1148,935,1014,1224,1834,...,2949,2709,2597,3116,4220,4626,level_7_CA_HOUSEHOLD_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
6,CA,HOUSEHOLD_2,765,867,576,630,631,593,708,920,...,898,930,822,973,1402,1398,level_7_CA_HOUSEHOLD_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
7,TX,FOODS_1,500,446,381,317,297,445,337,466,...,613,644,707,695,748,650,level_7_TX_FOODS_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
8,TX,FOODS_2,1384,1426,1008,1316,1143,1826,917,1319,...,1268,1168,1128,1115,1085,1341,level_7_TX_FOODS_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
9,TX,FOODS_3,4969,5158,3735,3837,3162,4796,3417,5270,...,5185,4430,4332,5029,6016,5774,level_7_TX_FOODS_3_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3


#### Level 8

In [121]:
level8 = create_level_all_products(train_df, ['store_id','cat_id'], '8')
level8

,store_id,cat_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id,item_id,dept_id,state_id
0,CA_1,FOODS,3239,3137,2008,2258,2032,2407,2693,3962,...,2488,2490,2533,2960,3655,4053,level_8_CA_1_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
1,CA_1,HOBBIES,556,498,415,392,268,467,346,769,...,495,363,410,527,670,714,level_8_CA_1_HOBBIES_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
2,CA_1,HOUSEHOLD,542,520,393,401,330,402,411,706,...,810,869,766,900,1252,1346,level_8_CA_1_HOUSEHOLD_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
3,CA_2,FOODS,2193,1921,1289,1540,1278,1494,1785,2385,...,2410,2506,2261,2871,3760,4012,level_8_CA_2_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
4,CA_2,HOBBIES,538,397,368,350,296,391,316,413,...,368,344,271,485,542,507,level_8_CA_2_HOBBIES_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
5,CA_2,HOUSEHOLD,763,728,464,434,368,403,528,931,...,853,841,771,1101,1582,1563,level_8_CA_2_HOUSEHOLD_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
6,CA_3,FOODS,3446,3535,2701,3064,2761,3340,3455,4068,...,3468,3286,3056,3409,4459,4698,level_8_CA_3_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
7,CA_3,HOBBIES,550,430,438,424,364,390,493,525,...,588,534,540,598,765,670,level_8_CA_3_HOBBIES_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
8,CA_3,HOUSEHOLD,743,862,646,744,692,639,755,863,...,1627,1415,1422,1616,2195,2353,level_8_CA_3_HOUSEHOLD_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI
9,CA_4,FOODS,1223,1269,946,1002,1107,1015,1072,1455,...,1722,1581,1629,1751,1971,2043,level_8_CA_4_FOODS_validation,all,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,CA_TX_WI


#### Level 9

In [122]:
level9 = create_level_all_products(train_df, ['store_id','dept_id'], '9')
level9

,store_id,dept_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id,item_id,cat_id,state_id
0,CA_1,FOODS_1,297,284,214,175,182,191,224,263,...,237,287,297,331,419,329,level_9_CA_1_FOODS_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
1,CA_1,FOODS_2,674,655,396,476,354,486,554,747,...,391,423,365,489,590,734,level_9_CA_1_FOODS_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
2,CA_1,FOODS_3,2268,2198,1398,1607,1496,1730,1915,2952,...,1860,1780,1871,2140,2646,2990,level_9_CA_1_FOODS_3_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
3,CA_1,HOBBIES_1,528,489,409,383,263,453,339,750,...,461,333,380,471,621,641,level_9_CA_1_HOBBIES_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
4,CA_1,HOBBIES_2,28,9,6,9,5,14,7,19,...,34,30,30,56,49,73,level_9_CA_1_HOBBIES_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,WI_3,FOODS_3,2293,2383,1841,1965,1427,2760,2727,3786,...,1715,1579,1670,1897,2411,2233,level_9_WI_3_FOODS_3_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
66,WI_3,HOBBIES_1,256,342,228,183,70,285,357,399,...,249,202,182,264,381,230,level_9_WI_3_HOBBIES_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
67,WI_3,HOBBIES_2,22,14,20,11,4,8,3,13,...,27,42,21,32,32,21,level_9_WI_3_HOBBIES_2_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI
68,WI_3,HOUSEHOLD_1,584,541,420,327,151,392,492,647,...,558,535,573,701,801,766,level_9_WI_3_HOUSEHOLD_1_validation,all,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI


#### Level 10

In [123]:
level10 = create_level_by_product(train_df, ['item_id'], '10')
level10

,item_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1909,d_1910,d_1911,d_1912,d_1913,id,dept_id,cat_id,state_id,store_id
0,FOODS_1_001,6,6,4,6,7,18,10,4,11,...,3,6,2,16,6,level_10_FOODS_1_001_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
1,FOODS_1_002,4,5,7,4,3,4,1,7,2,...,5,5,3,3,1,level_10_FOODS_1_002_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
2,FOODS_1_003,14,8,3,6,3,8,13,10,11,...,4,4,3,11,5,level_10_FOODS_1_003_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3,FOODS_1_004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,level_10_FOODS_1_004_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
4,FOODS_1_005,34,32,13,20,10,21,18,20,25,...,27,12,15,38,9,level_10_FOODS_1_005_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,HOUSEHOLD_2_512,5,4,1,3,2,4,2,8,7,...,2,8,6,10,5,level_10_HOUSEHOLD_2_512_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3045,HOUSEHOLD_2_513,0,0,0,0,0,0,0,0,0,...,4,6,7,4,11,level_10_HOUSEHOLD_2_513_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3046,HOUSEHOLD_2_514,4,8,2,1,1,2,3,8,2,...,0,0,2,2,2,level_10_HOUSEHOLD_2_514_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3047,HOUSEHOLD_2_515,0,0,0,0,0,0,0,0,0,...,1,1,1,5,1,level_10_HOUSEHOLD_2_515_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_TX_WI,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3


#### Level 11

In [124]:
level11 = create_level_by_product(train_df, ['item_id','state_id'], '11')
level11

,item_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,id,dept_id,cat_id,store_id
0,FOODS_1_001,CA,6,3,2,3,7,5,8,3,...,2,2,4,2,3,5,level_11_FOODS_1_001_CA_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
1,FOODS_1_001,TX,0,1,2,2,0,4,0,0,...,0,0,2,0,1,0,level_11_FOODS_1_001_TX_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
2,FOODS_1_001,WI,0,2,0,1,0,9,2,1,...,3,1,0,0,12,1,level_11_FOODS_1_001_WI_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
3,FOODS_1_002,CA,3,3,4,4,3,3,0,2,...,0,2,2,1,2,1,level_11_FOODS_1_002_CA_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
4,FOODS_1_002,TX,0,0,2,0,0,0,0,1,...,2,1,0,0,0,0,level_11_FOODS_1_002_TX_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,HOUSEHOLD_2_515,TX,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,level_11_HOUSEHOLD_2_515_TX_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
9143,HOUSEHOLD_2_515,WI,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,level_11_HOUSEHOLD_2_515_WI_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
9144,HOUSEHOLD_2_516,CA,0,2,0,1,1,1,0,2,...,0,0,1,1,0,1,level_11_HOUSEHOLD_2_516_CA_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3
9145,HOUSEHOLD_2_516,TX,2,1,0,0,0,0,1,1,...,0,1,0,0,0,0,level_11_HOUSEHOLD_2_516_TX_validation,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3


### Concatenatin all levels

In [125]:
new_sales = pd.concat([train_df,
                       level1,
                       level2,
                       level3,
                       level4,
                       level5,
                       level6,
                       level7,
                       level8,
                       level9,
                       level10,
                       level11], axis = 0)
new_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,level_11_HOUSEHOLD_2_515_TX_validation,HOUSEHOLD_2_515,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,TX,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9143,level_11_HOUSEHOLD_2_515_WI_validation,HOUSEHOLD_2_515,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,WI,0,0,0,0,...,0,1,0,0,0,1,1,1,1,0
9144,level_11_HOUSEHOLD_2_516_CA_validation,HOUSEHOLD_2_516,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,CA,0,2,0,1,...,3,0,0,0,0,0,1,1,0,1
9145,level_11_HOUSEHOLD_2_516_TX_validation,HOUSEHOLD_2_516,FOODS_1_FOODS_2_FOODS_3_HOBBIES_1_HOBBIES_2_HO...,FOODS_HOBBIES_HOUSEHOLD,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,TX,2,1,0,0,...,0,1,1,0,0,1,0,0,0,0


In [127]:
new_sales.to_csv('data/sales_train_validation_all_levels.csv', index = False)

## Sell Price Data

### Loading sell price df to create levels

In [7]:
prices_df = pd.read_csv('data/sell_prices.csv')

### Calculating Levels

In [26]:
prices_df['state_id'] = prices_df['store_id'].apply(lambda x: x.split('_')[0])
prices_df['dept_id'] = prices_df['item_id'].apply(lambda x: '_'.join(x.split('_')[:2]))
prices_df['cat_id'] = prices_df['item_id'].apply(lambda x: x.split('_')[0])

In [27]:
prices_df

,store_id,item_id,wm_yr_wk,sell_price,state_id,dept_id,cat_id
0,CA_1,HOBBIES_1_001,11325,9.58,CA,HOBBIES_1,HOBBIES
1,CA_1,HOBBIES_1_001,11326,9.58,CA,HOBBIES_1,HOBBIES
2,CA_1,HOBBIES_1_001,11327,8.26,CA,HOBBIES_1,HOBBIES
3,CA_1,HOBBIES_1_001,11328,8.26,CA,HOBBIES_1,HOBBIES
4,CA_1,HOBBIES_1_001,11329,8.26,CA,HOBBIES_1,HOBBIES
...,...,...,...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00,WI,FOODS_3,FOODS
6841117,WI_3,FOODS_3_827,11618,1.00,WI,FOODS_3,FOODS
6841118,WI_3,FOODS_3_827,11619,1.00,WI,FOODS_3,FOODS
6841119,WI_3,FOODS_3_827,11620,1.00,WI,FOODS_3,FOODS


In [32]:
prices_df.columns

Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price', 'state_id', 'dept_id',
       'cat_id'],
      dtype='object')

In [17]:
def create_level_sell_price(data, by, level_num):
    
    index_col = ['store_id','item_id','wm_yr_wk','sell_price']
    
    for col in by:
        index_col.remove(col)
    level = data.groupby(by).sum().reset_index()
    data_len = np.prod([data[col].unique().shape[0] for col in by])
    
    ids = [data[col].unique() for col in by]
    ids = list(itertools.product(*ids))
    ids = sorted(['_'.join(i) for i in ids])
    
    level[index_col] = pd.DataFrame([['level' + '_' + level_num + '_' + i + '_' + 'validation' for i in ids]] +
                                     [['_'.join(sorted(data[col].unique()))]*data_len for col in index_col[1:]
                                     ]).T
    return level

#### Level 1

In [31]:
level1 = prices_df.groupby(['wm_yr_wk']).sum().reset_index()
level1

,wm_yr_wk,sell_price
0,11101,44506.87
1,11102,49834.85
2,11103,52460.29
3,11104,53766.19
4,11105,54777.38
...,...,...
277,11617,136596.09
278,11618,136615.31
279,11619,136654.39
280,11620,136674.07


In [33]:
length = level1.shape[0]
length

282

In [34]:
level1[['store_id','item_id']] = pd.DataFrame([['_'.join(sorted(prices_df['store_id'].unique()))]*length] +
                                              [['all']*length]).T
level1

,wm_yr_wk,sell_price,store_id,item_id
0,11101,44506.87,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
1,11102,49834.85,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
2,11103,52460.29,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
3,11104,53766.19,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
4,11105,54777.38,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
...,...,...,...,...
277,11617,136596.09,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
278,11618,136615.31,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
279,11619,136654.39,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
280,11620,136674.07,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all


#### Level 2

In [35]:
level2 = prices_df.groupby(['wm_yr_wk','state_id']).sum().reset_index()
level2

,wm_yr_wk,state_id,sell_price
0,11101,CA,18551.72
1,11101,TX,13793.47
2,11101,WI,12161.68
3,11102,CA,20419.94
4,11102,TX,15528.87
...,...,...,...
841,11620,TX,40821.45
842,11620,WI,41112.35
843,11621,CA,54735.96
844,11621,TX,40792.76


In [37]:
length = level2.shape[0]
length

846

In [38]:
level2[['store_id','item_id']] = pd.DataFrame([['_'.join(sorted(prices_df['store_id'].unique()))]*length] +
                                              [['all']*length]).T
level2

,wm_yr_wk,state_id,sell_price,store_id,item_id
0,11101,CA,18551.72,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
1,11101,TX,13793.47,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
2,11101,WI,12161.68,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
3,11102,CA,20419.94,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
4,11102,TX,15528.87,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
...,...,...,...,...,...
841,11620,TX,40821.45,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
842,11620,WI,41112.35,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
843,11621,CA,54735.96,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all
844,11621,TX,40792.76,CA_1_CA_2_CA_3_CA_4_TX_1_TX_2_TX_3_WI_1_WI_2_WI_3,all


## Loading Data

In [7]:
########################### Vars
#################################################################################
TARGET = 'sales'         # Our main target
END_TRAIN = 1913         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

########################### Load Data
#################################################################################
print('Load Main Data')

Load Main Data


In [8]:
# Here are reafing all our data 
# without any limitations and dtype modification
train_df = pd.read_csv('data/sales_train_validation_all_levels.csv')
prices_df = pd.read_csv('data/sell_prices.csv')
calendar_df = pd.read_csv('data/calendar.csv')

## Processing Data

### Grid 1

In [9]:
########################### Make Grid
#################################################################################
print('Create Grid')

# We can tranform horizontal representation 
# to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

# If we look on train_df we se that 
# we don't have a lot of traning rows
# but each day can provide more train data
print('Train rows:', len(train_df), len(grid_df))

# To be able to make predictions
# we need to add "test set" to our grid
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

# Remove some temoprary DFs
del temp_df, add_grid

# We will not need original train_df
# anymore and can remove it
del train_df

# You don't have to use df = df construction
# you can use inplace=True instead.
# like this
# grid_df.reset_index(drop=True, inplace=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Create Grid
Train rows: 42840 81952920
    Original grid_df:   5.0GiB
     Reduced grid_df:   2.0GiB


In [10]:
########################### Product Release date
#################################################################################
print('Release week')

# It seems that leadings zero values
# in each train_df item row
# are not real 0 sales but mean
# absence for the item in the store
# we can safe some memory by removing
# such zeros

# Prices are set by week
# so it we will have not very accurate release week 
release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id','item_id','release']

# Now we can merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df

# We want to remove some "zeros" rows
# from grid_df 
# to do it we need wm_yr_wk column
# let's merge partly calendar_df to have it
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d'])
                      
# Now we can cutoff some rows 
# and safe memory 
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# Should we keep release week 
# as one of the features?
# Only good CV can give the answer.
# Let's minify the release values.
# Min transformation will not help here 
# as int16 -> Integer (-32768 to 32767)
# and our grid_df['release'].max() serves for int16
# but we have have an idea how to transform 
# other columns in case we will need it
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Release week
    Original grid_df:   1.8GiB
     Reduced grid_df:   1.6GiB


In [11]:
########################### Save part 1
#################################################################################
print('Save Part 1')

# We have our BASE grid ready
# and can save it as pickle file
# for future use (model training)
grid_df.to_pickle('grid_part_1.pkl')

print('Size:', grid_df.shape)

Save Part 1
Size: (46881677, 10)


### Grid 2

In [12]:
########################### Prices
#################################################################################
print('Prices')

# We can do some basic aggregations
prices_df['price_max'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# and do price normalization (min/max scaling)
prices_df['price_norm'] = prices_df['sell_price']/prices_df['price_max']

# Some items are can be inflation dependent
# and some items are very "stable"
prices_df['price_nunique'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

# I would like some "rolling" aggregations
# but would like months and years as "window"
calendar_prices = calendar_df[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices

# Now we can add price "momentum" (some sort of)
# Shifted by week 
# by month mean
# by year mean
prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

del prices_df['month'], prices_df['year']

Prices


In [13]:
########################### Merge prices and save part 2
#################################################################################
print('Merge prices and save part 2')

# Merge Prices
original_columns = list(grid_df)
grid_df = grid_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)

# Safe part 2
grid_df.to_pickle('grid_part_2.pkl')
print('Size:', grid_df.shape)

# We don't need prices_df anymore
del prices_df

# We can remove new columns
# or just load part_1
grid_df = pd.read_pickle('grid_part_1.pkl')

Merge prices and save part 2
Mem. usage decreased to 1879.39 Mb (61.1% reduction)
Size: (46881677, 13)


### Grid 3

In [14]:
########################### Merge calendar
#################################################################################
grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

# Minify data
# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

# Convert to DateTime
grid_df['date'] = pd.to_datetime(grid_df['date'])

# Make some features from date
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

# Remove date
del grid_df['date']

In [15]:
########################### Save part 3 (Dates)
#################################################################################
print('Save part 3')

# Safe part 3
grid_df.to_pickle('grid_part_3.pkl')
print('Size:', grid_df.shape)

# We don't need calendar_df anymore
del calendar_df
del grid_df

Save part 3
Size: (46881677, 16)


In [16]:
########################### Some additional cleaning
#################################################################################

## Part 1
# Convert 'd' to int
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk'
# as test values are not in train set
del grid_df['wm_yr_wk']
grid_df.to_pickle('grid_part_1.pkl')

del grid_df

### Loading Optimized Data

In [7]:
# Now we have 3 sets of features
grid_df = pd.concat([pd.read_pickle('grid_part_1.pkl'),
                     pd.read_pickle('grid_part_2.pkl').iloc[:,2:],
                     pd.read_pickle('grid_part_3.pkl').iloc[:,2:]],
                     axis=1)

In [8]:
# Let's check again memory usage
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
print('Size:', grid_df.shape)

           Full Grid:   2.5GiB
Size: (46881677, 34)


In [7]:
grid_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,...,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,0,0.459961,...,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,0,1.559570,...,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,3.169922,...,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,5.980469,...,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,0,0.700195,...,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1941,NaN,0,2.980469,...,0,0,0,22,20,5,5,4,6,1
46881673,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1941,NaN,0,2.480469,...,0,0,0,22,20,5,5,4,6,1
46881674,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1941,NaN,0,3.980469,...,0,0,0,22,20,5,5,4,6,1
46881675,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1941,NaN,230,1.280273,...,0,0,0,22,20,5,5,4,6,1


In [8]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46881677 entries, 0 to 46881676
Data columns (total 34 columns):
 #   Column            Dtype   
---  ------            -----   
 0   id                category
 1   item_id           category
 2   dept_id           category
 3   cat_id            category
 4   store_id          category
 5   state_id          category
 6   d                 int16   
 7   sales             float64 
 8   release           int16   
 9   sell_price        float16 
 10  price_max         float16 
 11  price_min         float16 
 12  price_std         float16 
 13  price_mean        float16 
 14  price_norm        float16 
 15  price_nunique     float16 
 16  item_nunique      int16   
 17  price_momentum    float16 
 18  price_momentum_m  float16 
 19  price_momentum_y  float16 
 20  event_name_1      category
 21  event_type_1      category
 22  event_name_2      category
 23  event_type_2      category
 24  snap_CA           category
 25  snap_TX         

In [13]:
grid_df.id.unique().shape[0] * 1941

59181090

### Generating Aggregation Levels

In [19]:
grid_df.item_id.value_counts().to_frame()

,item_id
FOODS_3_586,19410
HOBBIES_1_213,19410
HOBBIES_1_164,19410
FOODS_3_295,19410
HOBBIES_1_172,19410
...,...
HOUSEHOLD_1_308,4164
HOUSEHOLD_1_159,4031
HOUSEHOLD_1_242,3870
FOODS_3_296,3814


#### Level 1

In [26]:
grid_df.groupby(['d'])['sales'].sum().to_frame().reset_index()

,d,sales
0,1,32631.0
1,2,31749.0
2,3,23783.0
3,4,25412.0
4,5,19146.0
...,...,...
1936,1937,0.0
1937,1938,0.0
1938,1939,0.0
1939,1940,0.0


#### Level 2

In [23]:
grid_df.groupby(['state_id','d'])['sales'].sum().to_frame().reset_index()

,state_id,d,sales
0,CA,1,14195.0
1,CA,2,13805.0
2,CA,3,10108.0
3,CA,4,11047.0
4,CA,5,9925.0
...,...,...,...
5818,WI,1937,0.0
5819,WI,1938,0.0
5820,WI,1939,0.0
5821,WI,1940,0.0


#### Level 3

In [25]:
grid_df.groupby(['store_id','d'])['sales'].sum().to_frame().reset_index()

,store_id,d,sales
0,CA_1,1,4337.0
1,CA_1,2,4155.0
2,CA_1,3,2816.0
3,CA_1,4,3051.0
4,CA_1,5,2630.0
...,...,...,...
19405,WI_3,1937,0.0
19406,WI_3,1938,0.0
19407,WI_3,1939,0.0
19408,WI_3,1940,0.0


#### Level 4

In [27]:
grid_df.groupby(['cat_id','d'])['sales'].sum().to_frame().reset_index()

,cat_id,d,sales
0,FOODS,1,23178.0
1,FOODS,2,22758.0
2,FOODS,3,17174.0
3,FOODS,4,18878.0
4,FOODS,5,14603.0
...,...,...,...
5818,HOUSEHOLD,1937,0.0
5819,HOUSEHOLD,1938,0.0
5820,HOUSEHOLD,1939,0.0
5821,HOUSEHOLD,1940,0.0


#### Level 5

In [28]:
grid_df.groupby(['dept_id','d'])['sales'].sum().to_frame().reset_index()

,dept_id,d,sales
0,FOODS_1,1,2343.0
1,FOODS_1,2,2216.0
2,FOODS_1,3,1657.0
3,FOODS_1,4,1508.0
4,FOODS_1,5,1209.0
...,...,...,...
13582,HOUSEHOLD_2,1937,0.0
13583,HOUSEHOLD_2,1938,0.0
13584,HOUSEHOLD_2,1939,0.0
13585,HOUSEHOLD_2,1940,0.0


#### Level 6

In [30]:
1941 * 9

17469

In [31]:
grid_df.groupby(['state_id','cat_id','d'])['sales'].sum().to_frame().reset_index()

,state_id,cat_id,d,sales
0,CA,FOODS,1,10101.0
1,CA,FOODS,2,9862.0
2,CA,FOODS,3,6944.0
3,CA,FOODS,4,7864.0
4,CA,FOODS,5,7178.0
...,...,...,...,...
17464,WI,HOUSEHOLD,1937,0.0
17465,WI,HOUSEHOLD,1938,0.0
17466,WI,HOUSEHOLD,1939,0.0
17467,WI,HOUSEHOLD,1940,0.0


#### Level 7

In [32]:
1941 * 21

40761

In [33]:
grid_df.groupby(['state_id','dept_id','d'])['sales'].sum().to_frame().reset_index()

,state_id,dept_id,d,sales
0,CA,FOODS_1,1,1157.0
1,CA,FOODS_1,2,1142.0
2,CA,FOODS_1,3,787.0
3,CA,FOODS_1,4,743.0
4,CA,FOODS_1,5,729.0
...,...,...,...,...
40756,WI,HOUSEHOLD_2,1937,0.0
40757,WI,HOUSEHOLD_2,1938,0.0
40758,WI,HOUSEHOLD_2,1939,0.0
40759,WI,HOUSEHOLD_2,1940,0.0


#### Level 8

In [34]:
1941 * 30

58230

In [35]:
grid_df.groupby(['store_id','cat_id','d'])['sales'].sum().to_frame().reset_index()

,store_id,cat_id,d,sales
0,CA_1,FOODS,1,3239.0
1,CA_1,FOODS,2,3137.0
2,CA_1,FOODS,3,2008.0
3,CA_1,FOODS,4,2258.0
4,CA_1,FOODS,5,2032.0
...,...,...,...,...
58225,WI_3,HOUSEHOLD,1937,0.0
58226,WI_3,HOUSEHOLD,1938,0.0
58227,WI_3,HOUSEHOLD,1939,0.0
58228,WI_3,HOUSEHOLD,1940,0.0


#### Level 9

In [36]:
1941 * 70

135870

In [37]:
grid_df.groupby(['store_id','dept_id','d'])['sales'].sum().to_frame().reset_index()

,store_id,dept_id,d,sales
0,CA_1,FOODS_1,1,297.0
1,CA_1,FOODS_1,2,284.0
2,CA_1,FOODS_1,3,214.0
3,CA_1,FOODS_1,4,175.0
4,CA_1,FOODS_1,5,182.0
...,...,...,...,...
135865,WI_3,HOUSEHOLD_2,1937,0.0
135866,WI_3,HOUSEHOLD_2,1938,0.0
135867,WI_3,HOUSEHOLD_2,1939,0.0
135868,WI_3,HOUSEHOLD_2,1940,0.0


#### Level 10

In [14]:
1941 * 3049

5918109

In [ ]:
grid_df.groupby(['item_id','store_id','state_id','d'])['sales'].sum().to_frame().reset_index()

### Plots